In [3]:
from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans 
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim  
!conda install -c conda-forge folium --yes
!conda install -c conda-forge lxml --yes


from pandas.io.json import json_normalize 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import requests
import json
from bs4 import BeautifulSoup
import matplotlib.cm as cm
import matplotlib.colors as colors

%matplotlib inline



print("imports completed")

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

imports completed


In [4]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_areas_of_London').text
soup = BeautifulSoup(source, 'lxml')
article = soup.find('table', class_='wikitable sortable')



table_list = []
for rows in article.find_all('td'):
    row = rows.text
    row = row.replace('\n', '')
    table_list.append(row)

table_list

['Abbey Wood',
 'Bexley,  Greenwich [7]',
 'LONDON',
 'SE2',
 '020',
 'TQ465785',
 'Acton',
 'Ealing, Hammersmith and Fulham[8]',
 'LONDON',
 'W3, W4',
 '020',
 'TQ205805',
 'Addington',
 'Croydon[8]',
 'CROYDON',
 'CR0',
 '020',
 'TQ375645',
 'Addiscombe',
 'Croydon[8]',
 'CROYDON',
 'CR0',
 '020',
 'TQ345665',
 'Albany Park',
 'Bexley',
 'BEXLEY, SIDCUP',
 'DA5, DA14',
 '020',
 'TQ478728',
 'Aldborough Hatch',
 'Redbridge[9]',
 'ILFORD',
 'IG2',
 '020',
 'TQ455895',
 'Aldgate',
 'City[10]',
 'LONDON',
 'EC3',
 '020',
 'TQ334813',
 'Aldwych',
 'Westminster[10]',
 'LONDON',
 'WC2',
 '020',
 'TQ307810',
 'Alperton',
 'Brent[11]',
 'WEMBLEY',
 'HA0',
 '020',
 'TQ185835',
 'Anerley',
 'Bromley[11]',
 'LONDON',
 'SE20',
 '020',
 'TQ345695',
 'Angel',
 'Islington[8]',
 'LONDON',
 'EC1, N1',
 '020',
 'TQ345665',
 'Aperfield',
 'Bromley[11]',
 'WESTERHAM',
 'TN16',
 '01959',
 'TQ425585',
 'Archway',
 'Islington[12]',
 'LONDON',
 'N19',
 '020',
 'TQ285875',
 'Ardleigh Green',
 'Havering[12]',


In [5]:
location = table_list[::6]
Borough = table_list[1::6]
Neighbourhood = table_list[2::6]
PostalCode = table_list[3::6]
DialCode = table_list[4::6]
Gridref = table_list[5::6]

In [6]:
london_df = pd.DataFrame({"PostalCode": PostalCode,
                           "Borough": Borough,
                           "Neighbourhood": Neighbourhood,
                           "Location": location})

london_df.head()

,PostalCode,Borough,Neighbourhood,Location
0,SE2,"Bexley, Greenwich [7]",LONDON,Abbey Wood
1,"W3, W4","Ealing, Hammersmith and Fulham[8]",LONDON,Acton
2,CR0,Croydon[8],CROYDON,Addington
3,CR0,Croydon[8],CROYDON,Addiscombe
4,"DA5, DA14",Bexley,"BEXLEY, SIDCUP",Albany Park


In [7]:
london_df1 = london_df.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
london_df1.head()

,PostalCode,Borough,Neighbourhood,Location
0,BR1,Bromley,"BROMLEY, BROMLEY, BROMLEY","Plaistow, Sundridge, Widmore (also Widmore Green)"
1,BR1,Bromley[35],BROMLEY,Bromley
2,BR1,Lewisham,BROMLEY,Downham
3,BR2,Bromley,"BROMLEY, KESTON, KESTON, BROMLEY","Hayes, Keston, Leaves Green, Southborough"
4,BR3,Bromley,"BECKENHAM, BECKENHAM","Eden Park, Elmers End"


In [8]:
!conda install -c conda-forge geocoder --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    click-7.1.2                |     pyh9f0ad1d_0          64 KB  conda-forge
    future-0.18.2              |   py36h9f0ad1d_1         714 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         837 KB

The following NEW packages will be INSTALLED:

  click              conda-forge/noarch::click-7.1.2-pyh9f0ad1d_0
  decorator          conda-forge/noarch::decorator-4.4.2-py_0
  future             conda-forge/linux-64::future-0.18.2-py36h9f0ad1d_1
  geocoder       

In [9]:
import geocoder
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, London, United Kingdom'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [10]:

coords = [ get_latlng(neighborhood) for neighborhood in london_df1["Neighbourhood"].tolist() ]
coords

[[51.41135250000001, 0.00943875000000998],
 [51.431820424666206, -0.016565652066046606],
 [51.431820424666206, -0.016565652066046606],
 [51.40573000000006, 0.015060000000062246],
 [51.40725000000003, -0.02778999999992493],
 [51.431820424666206, -0.016565652066046606],
 [51.431820424666206, -0.016565652066046606],
 [51.622515618192935, -0.07906934545176653],
 [51.37474000000003, -0.009689999999977772],
 [51.37334000000004, 0.08969000000007554],
 [51.62062802478268, -0.07779022563569461],
 [51.37334000000004, 0.08969000000007554],
 [51.419710000000066, 0.06584000000003698],
 [51.372340000000065, -0.09944999999993342],
 [51.3818822537198, -0.10618533954103655],
 [51.362310000000036, -0.09337999999996782],
 [51.38772927523384, -0.15822391864868268],
 [51.31886000000003, -0.13914999999997235],
 [51.401585211274494, -0.11848072848584217],
 [51.338670000000036, -0.11672999999996136],
 [51.622832082632826, -0.08065551264637581],
 [51.44411240071716, 0.24863236306673384],
 [51.44411240071716, 0

In [11]:
column_names = ["PostalCode", "Borough", "Neighborhood","Location" "Latitude", "Longitude"]


In [12]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [13]:
london_df1['Latitude'] = df_coords['Latitude']
london_df1['Longitude'] = df_coords['Longitude']

In [14]:
london_df1.head()

,PostalCode,Borough,Neighbourhood,Location,Latitude,Longitude
0,BR1,Bromley,"BROMLEY, BROMLEY, BROMLEY","Plaistow, Sundridge, Widmore (also Widmore Green)",51.411353,0.009439
1,BR1,Bromley[35],BROMLEY,Bromley,51.431820,-0.016566
2,BR1,Lewisham,BROMLEY,Downham,51.431820,-0.016566
3,BR2,Bromley,"BROMLEY, KESTON, KESTON, BROMLEY","Hayes, Keston, Leaves Green, Southborough",51.405730,0.015060
4,BR3,Bromley,"BECKENHAM, BECKENHAM","Eden Park, Elmers End",51.407250,-0.027790


In [16]:
address = 'London'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [17]:
# create map of Toronto using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(london_df1['Latitude'], london_df1['Longitude'], london_df1['Borough'], london_df1['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_london)  
    
map_london

In [28]:
london_df1.to_csv("london_df1.csv", index=False)

In [18]:
CLIENT_ID = 'XNYYQ4KRPBLSZSGV5WM5A433JKIS20CQVNHEWBFKWDHJCF4U' # your Foursquare ID
CLIENT_SECRET = 'U35V5SEV5KYK3PHLFHVCORRMEMLKIWKH1SPDP0KMYJ021OWV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: XNYYQ4KRPBLSZSGV5WM5A433JKIS20CQVNHEWBFKWDHJCF4U
CLIENT_SECRET:U35V5SEV5KYK3PHLFHVCORRMEMLKIWKH1SPDP0KMYJ021OWV


In [19]:
Harrow_data = london_df1[london_df1['Borough'] == 'Harrow'].reset_index(drop=True)
Harrow_data.head()

,PostalCode,Borough,Neighbourhood,Location,Latitude,Longitude
0,HA1,Harrow,"HARROW, HARROW","Harrow, Harrow on the Hill",51.567891,-0.339483
1,"HA1, HA2",Harrow,HARROW,North Harrow,51.513180,-0.106980
2,HA2,Harrow,"HARROW, HARROW","South Harrow, West Harrow",51.567891,-0.339483
3,HA3,Harrow,"HARROW, HARROW","Harrow Weald, Wealdstone",51.567891,-0.339483
4,HA5,Harrow,"PINNER, PINNER, PINNER","Hatch End, Pinner, Rayners Lane",51.598295,-0.393611


In [20]:
address = 'Harrow, London'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Harrow are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Harrow are 51.5967688, -0.33727515543507014.


In [21]:
# create map of Manhattan using latitude and longitude values
map_Harrow = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Harrow_data['Latitude'], Harrow_data['Longitude'], Harrow_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Harrow)  
    
map_Harrow

In [23]:
Harrow_data.loc[0, 'Neighbourhood']
neighbourhood_latitude = Harrow_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = Harrow_data.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = Harrow_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of HARROW, HARROW are 51.567890615208455, -0.3394832478241024.


In [24]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=XNYYQ4KRPBLSZSGV5WM5A433JKIS20CQVNHEWBFKWDHJCF4U&client_secret=U35V5SEV5KYK3PHLFHVCORRMEMLKIWKH1SPDP0KMYJ021OWV&v=20180605&ll=51.567890615208455,-0.3394832478241024&radius=500&limit=100'

In [25]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eebdb3d02a1720022d0395f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Harrow on the Hill',
  'headerFullLocation': 'Harrow on the Hill, London',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 8,
  'suggestedBounds': {'ne': {'lat': 51.57239061970846,
    'lng': -0.33225722254755463},
   'sw': {'lat': 51.56339061070845, 'lng': -0.34670927310065014}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bf00723d4e4d13ae2c215a7',
       'name': 'The Dolls House',
       'location': {'address': 'High St',
        'lat': 51.57199193889008,
        'lng': -0.3385245537770827,
        'labeledLatLngs': [{'label': 'display',
         

In [26]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [27]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,The Dolls House,Café,51.571992,-0.338525
1,Cafe Cafe,Bar,51.570247,-0.339739
2,The White Horse,Gastropub,51.570284,-0.345356
3,The Old Etonian,French Restaurant,51.572104,-0.338282
4,Battels,Coffee Shop,51.571443,-0.338886


next bit

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
    

In [29]:
Harrow_venues = getNearbyVenues(names=Harrow_data['Neighbourhood'],
                                   latitudes=Harrow_data['Latitude'],
                                   longitudes=Harrow_data['Longitude']
                                  )


HARROW, HARROW
HARROW
HARROW, HARROW
HARROW, HARROW
PINNER, PINNER, PINNER
STANMORE


In [30]:
print(Harrow_venues.shape)
Harrow_venues.head()

(153, 7)


,Neighborhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"HARROW, HARROW",51.567891,-0.339483,The Dolls House,51.571992,-0.338525,Café
1,"HARROW, HARROW",51.567891,-0.339483,Cafe Cafe,51.570247,-0.339739,Bar
2,"HARROW, HARROW",51.567891,-0.339483,The White Horse,51.570284,-0.345356,Gastropub
3,"HARROW, HARROW",51.567891,-0.339483,The Old Etonian,51.572104,-0.338282,French Restaurant
4,"HARROW, HARROW",51.567891,-0.339483,Battels,51.571443,-0.338886,Coffee Shop


Check how many venues for each neighbourhood

In [31]:
Harrow_venues.groupby('Neighborhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
HARROW,100,100,100,100,100,100
"HARROW, HARROW",24,24,24,24,24,24
"PINNER, PINNER, PINNER",4,4,4,4,4,4
STANMORE,25,25,25,25,25,25


data above is venues for each neighbourhood in harrow

In [32]:
print('There are {} uniques categories.'.format(len(Harrow_venues['Venue Category'].unique())))

There are 62 uniques categories.


In [33]:
# one hot encoding
Harrow_onehot = pd.get_dummies(Harrow_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Harrow_onehot['Neighborhood'] = Harrow_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Harrow_onehot.columns[-1]] + list(Harrow_onehot.columns[:-1])
Harrow_onehot = Harrow_onehot[fixed_columns]

Harrow_onehot.head()

,Neighborhood,American Restaurant,Argentinian Restaurant,Asian Restaurant,Athletics & Sports,Bar,Beer Bar,Building,Burger Joint,Burrito Place,...,Supermarket,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,"HARROW, HARROW",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"HARROW, HARROW",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"HARROW, HARROW",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"HARROW, HARROW",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"HARROW, HARROW",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
Harrow_grouped = Harrow_onehot.groupby('Neighborhood').mean().reset_index()
Harrow_grouped

,Neighborhood,American Restaurant,Argentinian Restaurant,Asian Restaurant,Athletics & Sports,Bar,Beer Bar,Building,Burger Joint,Burrito Place,...,Supermarket,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,HARROW,0.01,0.01,0.01,0.00,0.000,0.01,0.02,0.01,0.01,...,0.00,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.03
1,"HARROW, HARROW",0.00,0.00,0.00,0.00,0.125,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,"PINNER, PINNER, PINNER",0.00,0.00,0.00,0.00,0.000,0.00,0.00,0.00,0.00,...,0.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,STANMORE,0.00,0.00,0.00,0.04,0.040,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.12,0.00,0.00,0.00


In [35]:
num_top_venues = 5

for hood in Harrow_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Harrow_grouped[Harrow_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----HARROW----
                venue  freq
0         Coffee Shop  0.11
1                 Pub  0.11
2  Italian Restaurant  0.07
3      Sandwich Place  0.07
4          Restaurant  0.03


----HARROW, HARROW----
                venue  freq
0  Italian Restaurant  0.25
1                Café  0.12
2   French Restaurant  0.12
3         Coffee Shop  0.12
4           Gastropub  0.12


----PINNER, PINNER, PINNER----
                    venue  freq
0             Karaoke Bar  0.25
1  Furniture / Home Store  0.25
2             Supermarket  0.25
3                    Park  0.25
4     American Restaurant  0.00


----STANMORE----
                venue  freq
0                Café  0.16
1            Bus Stop  0.16
2  Turkish Restaurant  0.12
3  Italian Restaurant  0.08
4                 Bar  0.04




In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Harrow_grouped['Neighborhood']

for ind in np.arange(Harrow_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Harrow_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,HARROW,Coffee Shop,Pub,Sandwich Place,Italian Restaurant,Japanese Restaurant,Hotel,Restaurant,Wine Bar,Falafel Restaurant,Building
1,"HARROW, HARROW",Italian Restaurant,Coffee Shop,French Restaurant,Pub,Bar,Gastropub,Café,Event Space,Falafel Restaurant,Fast Food Restaurant
2,"PINNER, PINNER, PINNER",Karaoke Bar,Furniture / Home Store,Supermarket,Park,Wine Bar,Convenience Store,Gym,Grocery Store,Gastropub,Garden
3,STANMORE,Café,Bus Stop,Turkish Restaurant,Italian Restaurant,Indian Restaurant,Lounge,Metro Station,Middle Eastern Restaurant,Convenience Store,Park


Cluster this data

In [45]:
# set number of clusters
kclusters = 4

Harrow_grouped_clustering = Harrow_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Harrow_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 1, 2], dtype=int32)

In [48]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Harrow_merged = Harrow_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Harrow_merged = Harrow_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Harrow_merged.head() # check the last columns!

ValueError: cannot insert Cluster Labels, already exists

In [42]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Harrow_merged['Latitude'], Harrow_merged['Longitude'], Harrow_merged['Neighbourhood'], Harrow_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

KeyError: 'Cluster Labels'